In [1]:
%matplotlib inline

## Load data

In [1]:
import json
import pandas as pd
import os

import numpy as np
import sys
# add the 'src' directory as the one where we can import from
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

import data_loader
from trainer import train_model1, train_model2

In [2]:
data_dir = '/home/projects/flickr90k_v2/'
batch_size = 32
augment = True
random_seed = 42
valid_size=0.1
shuffle=True
show_sample = False
num_workers=0
pin_memory=False

use_parallel = True

train_loader, valid_loader = data_loader.get_train_valid_loader(data_dir,
                                                                batch_size,
                                                                augment,
                                                                random_seed,
                                                                valid_size=0.1,
                                                                shuffle=True,
                                                                show_sample=show_sample,
                                                                num_workers=4,
                                                                pin_memory=False)

test_loader = data_loader.get_test_loader(data_dir,
                                          batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          pin_memory=False)

In [3]:
print("Training data size: {} \nValidation data size: {} \nTest data size: {}" \
      .format(len(train_loader), len(valid_loader), len(test_loader)))

Training data size: 2391 
Validation data size: 266 
Test data size: 157


In [4]:
from sklearn.preprocessing import LabelEncoder
unique_classes = ['bird', 'boat', 'cat', 'dog', 'flower', 'frog', 'jumbojet', 'mushroom', 'sportscar', 'tree']
le = LabelEncoder()
le.fit(unique_classes)
print(le.transform(unique_classes))

[0 1 2 3 4 5 6 7 8 9]


# To do

1. class weighting

        #### Not implemented
        from sklearn.utils import class_weight

        class_weights = class_weight.compute_class_weight('balanced',
                                                          np.unique(train_df['class'].values),
                                                          train_df['class'].values)
        print(class_weights)
        class_weights = dict(enumerate(class_weights))

        class_weights
2. Deal with greyscale images

## Prepare dataset class, setup network and training data

In [5]:
#image_datasets = {'train': train_dataset , 'test': valid_dataset}
dataloaders = {'train': train_loader, 'valid':  valid_loader, 'test':  test_loader}
dataset_sizes = {'train': len(train_loader), 'valid': len(valid_loader), 'test': len(test_loader)}

### Prepare convolutional network and freeze layers

In [7]:
n_class = len(unique_classes)
freeze_layers = True

## Load the model 
model_conv = torchvision.models.inception_v3(pretrained='imagenet')

## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
if freeze_layers:
    for i, param in model_conv.named_parameters():
        param.requires_grad = False

# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class)

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, child in model_conv.named_children():
    if "Conv2d_4a_3x3" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)

NameError: name 'torchvision' is not defined

In [8]:
n_class = len(unique_classes)
#model_ft = models.resnet18(pretrained=True)
model_ft = models.inception_v3(pretrained='imagenet')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, n_class)

NameError: name 'models' is not defined

# Train the model

In [8]:
import numpy as np

import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd

#from pytorch_utils import train_model, evaluate_model, evaluate_model_test

In [6]:
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

device = 0
num_epochs = 2
use_gpu = True
lr=0.001

In [ ]:
n_class = len(unique_classes)
#model_ft = models.resnet18(pretrained=True)
model_ft = models.inception_v3(pretrained='imagenet')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, n_class)

model_ft = model_ft.to(device)
model_ft.aux_logits=False

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)
optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_ft.parameters())), lr=lr, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
model_ft = train_model2(model_ft,
                        criterion,
                        dataloaders,
                        optimizer,
                        scheduler,
                        num_epochs=25,
                        use_gpu=True)



  0%|          | 0/2391 [00:00<?, ?it/s]

Epoch 0/24
----------




  0%|          | 1/2391 [00:01<1:05:07,  1.63s/it]

  0%|          | 2/2391 [00:02<58:52,  1.48s/it]  

  0%|          | 3/2391 [00:03<54:32,  1.37s/it]

  0%|          | 4/2391 [00:04<51:36,  1.30s/it]

  0%|          | 5/2391 [00:06<49:26,  1.24s/it]

  0%|          | 6/2391 [00:07<48:02,  1.21s/it]

  0%|          | 7/2391 [00:08<46:56,  1.18s/it]

  0%|          | 8/2391 [00:09<46:08,  1.16s/it]

  0%|          | 9/2391 [00:10<45:35,  1.15s/it]

  0%|          | 10/2391 [00:11<45:18,  1.14s/it]

  0%|          | 11/2391 [00:12<45:02,  1.14s/it]

  1%|          | 12/2391 [00:13<44:52,  1.13s/it]

  1%|          | 13/2391 [00:15<44:46,  1.13s/it]

  1%|          | 14/2391 [00:16<44:36,  1.13s/it]

  1%|          | 15/2391 [00:17<44:29,  1.12s/it]

  1%|          | 16/2391 [00:18<44:26,  1.12s/it]

  1%|          | 17/2391 [00:19<44:30,  1.12s/it]

  1%|          | 18/2391 [00:20<44:25,  1.12s/it]

  1%|          | 19/2391 [00:21<44:25,  1.12s/it]

  1%|          | 20/2391 [00:22<44

NameError: name 'dataset_sizes' is not defined

In [10]:
model_ft = train_model2(model_ft,
                       dataloaders, 
                       dataset_sizes,
                       criterion, 
                       optimizer,
                       scheduler,
                       use_gpu,
                       num_epochs,
                       alpha = 0.1)

  0%|          | 0/2391 [00:00<?, ?it/s]

Epoch 0/1
----------


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/THC/generic/THCTensorMath.cu:24

In [25]:
lr = 0.1
step_size = 100

use_gpu = True
epochs = 100

step_size=7
lr=0.001

if use_parallel:
    print("[Using all the available GPUs]")
    model_conv = nn.DataParallel(model_conv, device_ids=[0])

print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=lr, momentum=0.9)

print("[Creating Learning rate scheduler...]")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=step_size, gamma=0.1)

print("[Training the model begun ....]")
model_ft = train_model(model_conv, dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, use_gpu, num_epochs=epochs)


  0%|          | 0/2391 [00:00<?, ?it/s]

[Using all the available GPUs]
[Using CrossEntropyLoss...]
[Using small learning rate with momentum...]
[Creating Learning rate scheduler...]
[Training the model begun ....]
Epoch 0/99
----------


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

# Evaluate on the test set

In [ ]:


def evaluate_model(mode, model_conv, input_data_loc, input_shape, use_gpu, name):
    """A function which evaluates the model and outputs the csv of predictions of validation and train.
    mode:
    model_conv:
    input_data_loc:
    input_shape:
    use_gpu:
    Output:
    1) Prints score of train and validation (Write it to a log file)
    """
    print("[Evaluating the data in {}]".format(mode))
    data_loc = os.path.join(input_data_loc, mode)

    print("[Building dataloaders]")
    dataloaders, image_datasets = data_loader_predict(data_loc, input_shape, name)
    class_to_idx = image_datasets.class_to_idx
    imgs = [i[0] for i in image_datasets.imgs]
    print("total number of {} images: {}".format(mode, len(imgs)))
    original, predicted, probs = [], [], []
    for img, label in dataloaders:
        if use_gpu:
            inputs = Variable(img.cuda())
        else:
            inputs = Variable(img)
        bs, ncrops, c, h, w = inputs.data.size()
        output = model_conv(inputs.view(-1, c, h, w)) # fuse batch size and ncrops
        if type(output) == tuple:
            output, _ = output
        else:
            output = output
        outputs = torch.stack([nn.Softmax(dim=0)(i) for i in output])
        outputs = outputs.mean(0)
        _, preds = torch.max(outputs, 0)
        probs.append(outputs.data.cpu().numpy())
        original.extend(label.numpy())
        predicted.extend(preds.data.cpu().numpy())
    print("Accuracy_score {} : {} ".format(mode,  accuracy_score(original, predicted)))
    frame = pd.DataFrame(probs)
    frame.columns = ["class_{}".format(i) for i in frame.columns]
    frame["img_loc"] = imgs
    frame["original"] = original
    frame["predicted"] = predicted
    return frame, class_to_idx


def evaluate_model_test(mode, model_conv, test_input_data_loc, input_shape, use_gpu, name):
    dataloaders, image_datasets = data_loader_predict(test_input_data_loc, input_shape, name)
    imgs =[i[0] for i in image_datasets.imgs]
    print("total number of {} images: {}".format(mode, len(imgs)))
    predicted, probs = [], []
    for img, label in dataloaders:
        if use_gpu:
            inputs = Variable(img.cuda())
        else:
            inputs = Variable(img)
        bs, ncrops, c, h, w = inputs.data.size()
        output = model_conv(inputs.view(-1, c, h, w)) # fuse batch size and ncrops
        if type(output) == tuple:
            output, _ = output
        else:
            output = output
        outputs = torch.stack([nn.Softmax(dim=0)(i) for i in output])
        outputs = outputs.mean(0)
        _, preds = torch.max(outputs, 0)
        probs.append(outputs.data.cpu().numpy())
        predicted.extend(preds.data.cpu().numpy())
    frame = pd.DataFrame(probs)
    frame.columns = ["class_{}".format(i) for i in frame.columns]
    frame["img_loc"] = imgs
    frame["predicted"] = predicted
    return frame

In [ ]:
test_dataset = Flickr90kDataset(test_df,
                                image_dir,
                                data_transforms['val'])

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)